<a href="https://colab.research.google.com/github/singh728om/IndigoHackToHire/blob/main/IndigoHackToHire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Step 1: Load and explore the dataset
dataset = load_dataset('toughdata/quora-question-answer-dataset')
print(dataset)

# Check for the splits in the dataset
if 'validation' not in dataset:
    # Assuming the train split needs to be split into train and validation
    train_valid = dataset['train'].train_test_split(test_size=0.1)  # 10% of train data for validation
    dataset = DatasetDict({
        'train': train_valid['train'],
        'validation': train_valid['test']
    })
print(dataset)

# Assuming the correct columns are 'question' and 'answer_text'
question_col = 'question'  # Replace with actual column name if different
context_col = 'answer'  # Replace with actual column name if different

# Step 2: Preprocess the data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(examples):
    questions = examples[question_col]
    contexts = examples[context_col]
    inputs = tokenizer(questions, contexts, truncation=True, padding=True)
    return inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Step 3: Model selection
model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

# Step 4: Training
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation']
)

# Train the model
trainer.train()

# Step 5: Evaluation
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Step 6: Save the model
model.save_pretrained('./qa_model')
tokenizer.save_pretrained('./qa_model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56402
    })
})
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 50761
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 5641
    })
})


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/50761 [00:00<?, ? examples/s]

Map:   0%|          | 0/5641 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

# Distribution of question lengths
question_lengths = dataset['train']['question'].apply(lambda x: len(x.split()))
sns.histplot(question_lengths, bins=30, kde=True)
plt.title('Distribution of Question Lengths')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()

# Distribution of answer lengths
answer_lengths = dataset['train']['answer'].apply(lambda x: len(x.split()))
sns.histplot(answer_lengths, bins=30, kde=True)
plt.title('Distribution of Answer Lengths')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()


NameError: name 'dataset' is not defined

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import datasets

# Load Dataset
data = datasets.load_dataset('toughdata/quora-question-answer-dataset')

# Data Exploration
print(data['train'].features)
print(data['train'][0])

# Data Cleaning and Preprocessing
# Tokenization, stop word removal, and stemming/lemmatization
def preprocess_text(text):
    # Implement tokenization, stop word removal, and stemming/lemmatization
    return text

data['train'] = data['train'].map(lambda x: {'question': preprocess_text(x['question']), 'answer': preprocess_text(x['answer'])})

# Model Selection and Evaluation
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['question'], examples['answer'], padding="max_length", truncation=True)

tokenized_datasets = data['train'].map(tokenize_function, batched=True)

# Train-Test Split
train_dataset, eval_dataset = train_test_split(tokenized_datasets, test_size=0.2)

# Training and Evaluation
model.compile(optimizer='adam', loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset, validation_data=eval_dataset, epochs=3)

# Evaluate the model
predictions = model.predict(eval_dataset)
pred_labels = np.argmax(predictions.logits, axis=-1)
true_labels = eval_dataset['label']

f1 = f1_score(true_labels, pred_labels, average='weighted')
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Visualization
sns.histplot(data['train']['question'], kde=True)
plt.title("Question Distribution")
plt.show()

# Insights and Recommendations
# Add insights and recommendations based on analysis

# Save the Model
model.save_pretrained("quora_question_answering_model")
tokenizer.save_pretrained("quora_question_answering_model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/api/datasets/toughdata/quora-question-answer-dataset (Request ID: Root=1-66a517ba-48ce7f375d69c5885f50ef45;c96f2e89-4b4a-4dc3-957b-0433ad396489)

Internal Error - We're working hard to fix this as soon as possible!